In [1]:
%matplotlib inline
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
batch_size = 256
num_workers = 4
#下载训练集到指定目录
mnist_train = torchvision.datasets.FashionMNIST(root='/home/retoo/Desktop/实验/数据集/3.深度学习/2FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
# 下载测试集到指定目录
mnist_test = torchvision.datasets.FashionMNIST(root='/home/retoo/Desktop/实验/数据集/3.深度学习/2FashionMNIST', train=False, download=True,transform=transforms.ToTensor())
# 将训练集组成一个批次
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
# 将测试集组成一个批次
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=num_workers)

In [3]:
feature, label = mnist_train[0]#读取第一个获得的训练数据，每一个训练数据均为data与label
print(feature.shape, label)  #每一个data为三通道数据,label为单通道标签数据

torch.Size([1, 28, 28]) 9


In [4]:
def relu(X):#定义了激活函数
    return torch.max(input=X, other=torch.tensor(0.0))

In [5]:
def softmax(X):#通过softmax函数，计算概率
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

In [6]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_hiddens)), dtype=torch.float)#采用初始化参数
b1 = torch.zeros(num_hiddens, dtype=torch.float)#采用零值初始化偏差
W2 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_outputs)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)#采用零值初始化偏差

params = [W1, b1, W2, b2]#参数所构成的列表
for param in params:
    param.requires_grad_(requires_grad=True)#设置所有的参数列表中的参数均需要进行反向求导


def net(X):#定义网络结构
    X = X.view((-1, num_inputs))#将输入的图像数据展平为单列的784的数据，有利于全连接层的使用
    H = relu(torch.matmul(X, W1) + b1)#将Ｈ＝X*W1+b1，再求激活函数
    O = softmax(torch.matmul(H, W2) + b2)#将Ｏ＝Ｈ*Ｗ２＋b2，再用softmax进行分类
    return O

In [7]:
def cross_entropy(y_hat, y):#定义交叉熵损失函数
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))
    #tensor.gather与torch.gather用法一样，
    # tensor.gather(dim,indexs)，在dim维度上，安装indexs所给的坐标选择元素，返回一个和index维度相同的tensor

In [8]:
def sgd(params, lr, batch_size):#参数优化方法
    for param in params:#所有需要优化的参数
        param.data -= lr * param.grad / batch_size # 由于param是tensor格式，因此更改param参数数据时用的param.data

In [9]:
def evaluate_accuracy(data_iter, net):#精度评价
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()#dim=1表示按行，取概率最大的，由于一次X包括了batch个数据，
        # 因此，把数据求和．.item()取出单元素张量的元素值并返回该值，保持原元素类型不变
        n += y.shape[0]
    return acc_sum / n

In [10]:
num_epochs, lr = 5, 0.1#定义训练的次数与学习率

def train_ch2(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0#每次训练对上次训练的结果进行归零
        for X, y in train_iter:
            y_hat = net(X)#
            l = loss(y_hat, y).sum()#l要求偏导，是tensor格式

            # 梯度清零
            if optimizer is not None:#如果定义了特定的参数优化器，则将参数优化器中的参数设置为０．本部分在使用的时候由于没有定义，所以不会执行下面的清零
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:#如果参数有定义并且参数第一个值的梯度也有定义，则执行下面的清零
                for param in params:
                    param.grad.data.zero_()

            l.backward()#梯度反向求导
            if optimizer is None:
                sgd(params, lr, batch_size)#没有定义特殊的优化器，则使用默认的SGD,本实验采用的是sgd
            else:
                optimizer.step()#定义了特殊的优化器，则直接用优化器step进行参数的学习


            train_l_sum += l.item()#把所有损失函数的值相加，即batch=256个值相加起来
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()#精度是由预测出的概率最大项与标签之间对比相同，
            n += y.shape[0]#即为batch值
        test_acc = evaluate_accuracy(test_iter, net)#验证数据集的精度，求解的方法与训练的数据集方法一样
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))#输出训练的次数，平均损失函数的大小，训练精度，测试精度

In [ ]:
train_ch2(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, params, lr)

epoch 1, loss 1.0443, train acc 0.645, test acc 0.737
epoch 2, loss 0.5993, train acc 0.789, test acc 0.799
epoch 3, loss 0.5196, train acc 0.817, test acc 0.822
